In [1]:
import os
import gc
import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal, osr
from tqdm import tqdm
import rasterio
from rasterstats import zonal_stats

In [ ]:
nuts_folder = '/data/xiang/1-Data/NUTS/nuts version'
read_folder = '/data/xiang/1-Data/PM2.5/raster'
# for nuts_file in os.listdir(nuts_folder):
nuts_list = ['NUTS_RG_01M_2006_3035.shp', 'NUTS_RG_01M_2013_3035.shp', 'NUTS_RG_01M_2016_3035.shp', 'NUTS_RG_01M_2003_3035.shp']
for nuts_file in nuts_list:
    df_comb = None
    if nuts_file.endswith('.shp'):
        nuts_file = nuts_folder + r'/' + nuts_file
        nuts = gpd.read_file(nuts_file)
        for PM_file in tqdm(os.listdir(read_folder)):
            read_file = read_folder + r'/' + PM_file
            year = PM_file.split('.')[-3][:4]
            # link the zonal statistics table to the boundary index
            zs_temp = pd.DataFrame(zonal_stats(nuts_file, read_file))['mean']
            df_temp = pd.merge(nuts[['NUTS_ID']], zs_temp, left_index=True, right_index=True)
            df_temp.columns = ['geo','ObsValue']
            df_temp['Indicator'] = 'PM2.5'
            df_temp['freq'] = '1 year'
            df_temp['ObsTime'] = year
            df_temp['unit'] = 'ug/m3'
            df_comb = df_temp if df_comb is None else pd.concat([df_comb, df_temp])
        # save the table
        geo_source = 'NUTS' + nuts_file.split('_')[-2]
        df_comb['geo_source'] = geo_source
        df_comb.set_index('geo').to_csv('/data/xiang/3-case studies/2-PM2.5/PM25_' + geo_source + '.csv')    
        # delete nuts and release the RAM
        del nuts
        gc.collect()